In [5]:
from mlflow import set_tracking_uri
import json
from typing import Dict

from hcve_lib.tracking import get_children_runs
from hcve_lib.tracking import get_completed_runs
from hcve_lib.tracking import is_root_run
from hcve_lib.visualisation import h3, h1, h4

set_tracking_uri('http://localhost:5000')

def get_lco_hyperparameters(method_name: str, experiment_name: str) -> Dict[str, Dict]:
    runs_lco = get_completed_runs(experiment_name)
    root_lco_runs = runs_lco[is_root_run(runs_lco)]
    coxnet_run = root_lco_runs.query(f'`tags.mlflow.runName` == "{method_name}"').iloc[0]
    children_runs = get_children_runs(runs_lco, coxnet_run['run_id'])
    return {
        rows['tags.mlflow.runName']: json.loads(rows['params.hyperparameters'].replace("\'", "\""))
        for _, rows in children_runs.iterrows()
    }


# LCO

In [ ]:
for method_name in 'coxnet', 'gb',:
    hyperparameters_per_cohort = get_lco_hyperparameters(method_name, 'optimized_lco')
    h1(method_name)
    h3('JSON')
    print(hyperparameters_per_cohort)

    h3('Per cohort')
    for cohort_name, hyperparameters in hyperparameters_per_cohort.items():
        h4(cohort_name)
        print(hyperparameters)

# Reproduce

In [9]:
for method_name in ('dcm',):
    hyperparameters_per_cohort = get_lco_hyperparameters(method_name, 'optimized_reproduce')
    h1(method_name)
    h3('JSON')
    print(hyperparameters_per_cohort)

    h3('Per cohort')
    for cohort_name, hyperparameters in hyperparameters_per_cohort.items():
        h4(cohort_name)
        print(hyperparameters)

{'train_test_filter': {'estimator': {'k': 4, 'layers': [42]}}}


{'estimator': {'k': 4, 'layers': [42]}}
